# Phase 1: Data Extraction & Database Migration

**Objective:** Fetch Clash Royale card data from the official API and load it into a raw SQL table.

In [ ]:
import requests
import pandas as pd
import sys
import os
from sqlalchemy import create_engine

# Add the parent directory to sys.path to import from src
sys.path.append(os.path.abspath(os.path.join('..')))

from src import config

## 1. Fetch Data from API

In [ ]:
def fetch_cards():
    headers = {
        "Accept": "application/json",
        "Authorization": f"Bearer {config.API_TOKEN}"
    }
    
    print(f"Fetching from {config.API_URL}...")
    response = requests.get(config.API_URL, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        return data.get('items', [])
    else:
        print(f"Error {response.status_code}: {response.text}")
        return []

cards_data = fetch_cards()
print(f"Fetched {len(cards_data)} cards.")

## 2. Prepare Data for Database
We will convert the JSON list to a Pandas DataFrame and flatten specific columns like `iconUrls`.

In [ ]:
if cards_data:
    df = pd.DataFrame(cards_data)
    
    # Flatten 'iconUrls' to extract just the medium icon URL
    if 'iconUrls' in df.columns:
        df['icon_medium'] = df['iconUrls'].apply(lambda x: x.get('medium') if isinstance(x, dict) else None)
        df.drop(columns=['iconUrls'], inplace=True)
        
    # Display first few rows
    print(df.head())

## 3. Upload to Database
This step requires a running PostgreSQL database.
**Constraint:** We upload to `raw_cards`. Cleaning happens inside the DB later.

In [ ]:
try:
    # Connect to DB
    engine = create_engine(config.get_db_connection_string())
    
    # Upload
    print("Uploading to database...")
    df.to_sql('raw_cards', con=engine, if_exists='replace', index=False)
    print("Success! Data loaded into 'raw_cards'.")
    
except Exception as e:
    print("Database Error:", e)
    print("\n[Action Required] Please ensure PostgreSQL is running and the database 'clashroyale_db' exists.")